In [ ]:
data = {'stream': 'xrpusdt@depth', 'data': {'e': 'depthUpdate', 'E': 1741231591317, 's': 'XRPUSDT', 'U': 16936333603, 'u': 16936334129, 'b': [['2.52490000', '0.00000000'], ['2.52480000', '5400.00000000'], ['2.52470000', '11730.00000000'], ['2.52460000', '9550.00000000'], ['2.52450000', '9518.00000000'], ['2.52440000', '5764.00000000'], ['2.52430000', '17410.00000000'], ['2.52420000', '15079.00000000'], ['2.52410000', '4467.00000000'], ['2.52400000', '8644.00000000'], ['2.52390000', '2244.00000000'], ['2.52380000', '11515.00000000'], ['2.52370000', '20067.00000000'], ['2.52360000', '18112.00000000'], ['2.52350000', '9389.00000000'], ['2.52320000', '3432.00000000'], ['2.52190000', '5481.00000000'], ['2.52180000', '3258.00000000'], ['2.52170000', '4876.00000000'], ['2.52100000', '13797.00000000'], ['2.52050000', '14308.00000000'], ['2.52010000', '6946.00000000'], ['2.52000000', '12823.00000000'], ['2.51980000', '18323.00000000'], ['2.51970000', '4979.00000000'], ['2.51960000', '6383.00000000'], ['2.51950000', '14799.00000000'], ['2.51920000', '15195.00000000'], ['2.51900000', '4953.00000000'], ['2.51720000', '8.00000000'], ['2.51580000', '1485.00000000'], ['2.51550000', '2138.00000000'], ['2.51480000', '2670.00000000'], ['2.51280000', '4685.00000000']], 'a': [['2.52490000', '5519.00000000'], ['2.52500000', '231.00000000'], ['2.52510000', '33.00000000'], ['2.52520000', '6296.00000000'], ['2.52530000', '4346.00000000'], ['2.52540000', '10953.00000000'], ['2.52550000', '12950.00000000'], ['2.52560000', '10776.00000000'], ['2.52570000', '7053.00000000'], ['2.52580000', '12583.00000000'], ['2.52590000', '7477.00000000'], ['2.52600000', '4974.00000000'], ['2.52610000', '6074.00000000'], ['2.52620000', '11932.00000000'], ['2.52630000', '3946.00000000'], ['2.52640000', '5618.00000000'], ['2.52650000', '6070.00000000'], ['2.52670000', '3628.00000000'], ['2.52680000', '7004.00000000'], ['2.52690000', '16174.00000000'], ['2.52700000', '16638.00000000'], ['2.52710000', '19881.00000000'], ['2.52730000', '3124.00000000'], ['2.52740000', '3501.00000000'], ['2.52760000', '3197.00000000'], ['2.52770000', '6266.00000000'], ['2.52780000', '9466.00000000'], ['2.52800000', '21262.00000000'], ['2.52810000', '20900.00000000'], ['2.52830000', '19250.00000000'], ['2.52840000', '13439.00000000'], ['2.52850000', '18366.00000000'], ['2.52860000', '9924.00000000'], ['2.52980000', '16302.00000000'], ['2.53010000', '30090.00000000'], ['2.53020000', '3505.00000000'], ['2.53030000', '7082.00000000'], ['2.53090000', '4343.00000000'], ['2.53120000', '2552.00000000'], ['2.53170000', '5083.00000000'], ['2.53180000', '10123.00000000'], ['2.53300000', '4057.00000000'], ['2.53310000', '3351.00000000'], ['2.53410000', '2103.00000000'], ['2.54070000', '6120.00000000'], ['2.54120000', '4046.00000000'], ['2.54150000', '4343.00000000'], ['2.54330000', '618.00000000']]}}

In [13]:
from Workspace.DataStorage.DataCollector.ExecutionStorage import ExecutionStorage
obj = ExecutionStorage()

message = {'e': 'ORDER_TRADE_UPDATE', 'T': 1741238353759, 'E': 1741238353759, 'o': {'s': 'BTCUSDT', 'c': 'ios_2h3rvaSHWpgYNznDHwpc', 'S': 'BUY', 'o': 'LIMIT', 'f': 'GTC', 'q': '11', 'p': '0.49980', 'ap': '0', 'sp': '0', 'x': 'CANCELED', 'X': 'CANCELED', 'i': 51472815751, 'l': '0', 'z': '0', 'L': '0', 'n': '0', 'N': 'USDT', 'T': 1741238353759, 't': 0, 'b': '0', 'a': '0', 'm': False, 'R': False, 'wt': 'CONTRACT_PRICE', 'ot': 'LIMIT', 'ps': 'BOTH', 'cp': False, 'rp': '0', 'pP': False, 'si': 0, 'ss': 0, 'V': 'EXPIRE_MAKER', 'pm': 'NONE', 'gtd': 0}}

In [14]:
obj.set_data(message)

In [15]:
obj.get_data('BTCUSDT', 'ORDER_TRADE_UPDATE')

{'e': 'ORDER_TRADE_UPDATE',
 'T': 1741238353759,
 'E': 1741238353759,
 'o': {'s': 'BTCUSDT',
  'c': 'ios_2h3rvaSHWpgYNznDHwpc',
  'S': 'BUY',
  'o': 'LIMIT',
  'f': 'GTC',
  'q': '11',
  'p': '0.49980',
  'ap': '0',
  'sp': '0',
  'x': 'CANCELED',
  'X': 'CANCELED',
  'i': 51472815751,
  'l': '0',
  'z': '0',
  'L': '0',
  'n': '0',
  'N': 'USDT',
  'T': 1741238353759,
  't': 0,
  'b': '0',
  'a': '0',
  'm': False,
  'R': False,
  'wt': 'CONTRACT_PRICE',
  'ot': 'LIMIT',
  'ps': 'BOTH',
  'cp': False,
  'rp': '0',
  'pP': False,
  'si': 0,
  'ss': 0,
  'V': 'EXPIRE_MAKER',
  'pm': 'NONE',
  'gtd': 0}}

In [ ]:
data.__slots__

In [3]:
class MyClass:
    def __new__(cls, name):
        print("__new__ Method create an instance!!")
        new_instance = [1,2,3,4,5]
        return name

    def __init__(self, name):
        print("__init__ Method doesn't create an instance...")
        self.name = name

my_instance = MyClass("init_mumu")
print(type(my_instance), my_instance)

__new__ Method create an instance!!
<class 'str'> init_mumu


In [4]:
my_instance

'init_mumu'